### Import Stuff

In [19]:
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
import pyspark.sql.types as T

### Create SQL Context

In [20]:
sqlContext = SQLContext(sc)

### Create Dataframes on Cassandra tables

In [21]:
stores_df = sqlContext.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(keyspace="retail", table="stores")\
    .load()

In [22]:
receipts_by_store_date_df = sqlContext.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(keyspace="retail", table="receipts_by_store_date")\
    .load()

### Save it

In [23]:
receipts_by_store_date_df.take(1)[0].customer.zip


u'87102'

In [24]:
receipts_by_store_date_df.show()
receipts_by_store_date_df.printSchema()



+--------+--------------------+-----------+--------------------+-------------+--------------------+------------------+
|store_id|        receipt_date|register_id|   receipt_timestamp|   receipt_id|            customer|     receipt_total|
+--------+--------------------+-----------+--------------------+-------------+--------------------+------------------+
|     145|2015-08-03 00:00:...|          1|2015-08-03 19:43:...|1444884264828|[Dustin,Baldwin,1...|            549.89|
|     145|2015-08-03 00:00:...|          2|2015-08-04 00:07:...|1444884156167|[Ethan,Lucas,422 ...|           1279.81|
|     145|2015-08-03 00:00:...|          5|2015-08-03 23:34:...|1444884235078|[Drew,Le,456 Sall...|           1299.84|
|     145|2015-08-03 00:00:...|          7|2015-08-03 16:58:...|1444884278936|[Kris,Workman,630...|            979.89|
|     145|2015-08-03 00:00:...|          7|2015-08-03 20:28:...|1444883394452|[Alex,Ramos,1058 ...|           1539.74|
|     145|2015-08-03 00:00:...|          7|2015-

In [25]:
receipts_by_store_date_df.rdd.map( lambda (store_id, receipt_date, register_id, receipt_timestamp, receipt_id, customer, receipt_total) : ((store_id, customer[5]),(receipt_total, 1))).reduceByKey(lambda x, y: x+y).sortBy(lambda (x,y): -y[0]).take(100)




[((200, u'66314'), (Decimal('4149.61'), 1)),
 ((394, u'54908'), (Decimal('3969.6699999999996'), 1)),
 ((91, u'47375'), (Decimal('3929.65'), 1)),
 ((165, u'27540'), (Decimal('3889.63'), 1)),
 ((292, u'59154'), (Decimal('3829.64'), 1, Decimal('1459.82'), 1)),
 ((131, u'01719'), (Decimal('3819.66'), 1)),
 ((134, u'79521'), (Decimal('3769.62'), 1)),
 ((136, u'91611'), (Decimal('3729.6299999999997'), 1)),
 ((11, u'87679'), (Decimal('3699.62'), 1)),
 ((90, u'85758'), (Decimal('3689.65'), 1)),
 ((204, u'44078'), (Decimal('3689.63'), 1)),
 ((330, u'00169'), (Decimal('3679.6400000000003'), 1)),
 ((103, u'56889'), (Decimal('3669.67'), 1)),
 ((114, u'44414'), (Decimal('3669.6'), 1)),
 ((51, u'89761'), (Decimal('3649.6299999999997'), 1)),
 ((126, u'24942'), (Decimal('3639.6499999999996'), 1, Decimal('1549.75'), 1)),
 ((100, u'78472'), (Decimal('3619.7'), 1)),
 ((199, u'11130'), (Decimal('3619.68'), 1)),
 ((99, u'25485'), (Decimal('3609.6499999999996'), 1)),
 ((376, u'16672'), (Decimal('3599.66'), 